![iceberg-logo](https://www.apache.org/logos/res/iceberg/iceberg.png)

### [CDC, Spark, and Iceberg: The Resilinet Way to Lakehouse!](https://tabular.io/blog/docker-spark-and-iceberg/)

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Jupyter").getOrCreate()

spark

24/07/20 14:15:52 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


### CDC data

For this notebook, we will use a custom data generator to showcase the problems that happens while mirroring source tables in data lakes. This function creates data for a sample dimension table for a data product company.

### Installing dependencies 
faker is a library that generates random data for such usecases

In [4]:
%pip install faker


[notice] A new release of pip is available: 23.0.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Imports and constants 

Paths are defined here

In [5]:
from pyspark.sql import Row, SparkSession
from pyspark.sql.functions import col, lit, current_timestamp, expr
from datetime import datetime as dt
from faker import Faker
from uuid import uuid1

fake = Faker()
GLOBAL_TEST_PATH = 'tests/test_data'
DATA_PATHS = {
    'DMS': {
        'SOURCE': f'{GLOBAL_TEST_PATH}/db_data',
        'SINK': f'{GLOBAL_TEST_PATH}/dms_sink'
    }
}
TABLES = {
    1: {
        'name': 'user_dimension',
    }
}


### Helper functions

These functions are going to help up create data in parquet and load them in iceberg

1. create_mock_dimension_table_dataframe
2. get_location
3. extract_parquet
4. load_parquet
5. load_table
6. load_mock_db_initial_state
7. load_mock_dms_full_load
8. load_mock_dms_cdc_load
9. load_mock_dms_cdc_d_load


In [15]:
def create_mock_dimension_table_dataframe(spark, row_count=10):
    """ Create initial test data for dimension table
    This function creates both pre- and post- transformation data
    saved as Parquet files in tests/test_data. This will be used for
    unit tests as well as to load as a part of example ingestion job
    :return: None
    """
    ts = dt.now()
    df = spark.range(row_count)
    local_records = [
        Row(
            name=fake.name(),
            address=fake.address(),
            license_number=fake.license_plate(),
            iban=fake.iban(),
            bs=fake.bs(),
            catch_phrase=fake.catch_phrase(),
            company=fake.company(),
            paragraph=fake.paragraph(nb_sentences=10),
            created_at=fake.date_time(),
            updated_at=ts
        )
        for i in range(row_count)
    ]
    df = spark.createDataFrame(local_records)
    df = df.withColumn("uuid", expr("uuid()"))

    print(f"""
        dataframe of count {row_count} 
        was created from faker as {ts}
    """)
    return df


In [16]:
def get_location(actor, io, table_name, action):
    """ Get Location
    <actors>/<table_name>/<action>/<timestamp>

    :params actor: executor that called the function
    :params io: intended SOURCE or SINK
    :params table_name: name of that table that the location
        is associated to
    :params action: key event that caused the data creation
    :return: Location
    """
    ts_str = dt.now().strftime("%Y_%m_%d-%I:%M:%S_%p")
    loc = f'{DATA_PATHS[actor][io]}/{table_name}/{action}/{ts_str}'
    print(f'''
        Location: {loc}
            for {actor}
            on {io} 
            for {table_name} 
            on {action}
            at {ts_str}
    ''')
    return loc

def extract_parquet(spark, loc):
    """Load data from Parquet file format.

    :param spark: Spark session object.
    :param loc: Location of data
    :return: Spark DataFrame.
    """
    df = (
        spark
        .read
        .parquet(loc))
    print(f"""
        dataframe of count {df.count()} '
        was read from location {loc}'
    """)
    return df


def load_parquet(df, loc, num_of_output_files=1):
    """Load data to Parquet file format.

    :param df: Spark Dataframe
    :param loc: Location of data
    :param num_of_output_files: number of files
    :return: None
    """

    (df
     .coalesce(num_of_output_files)
     .write
     .parquet(loc, mode='overwrite')
     )

    print(f'''
        Successfully written {num_of_output_files} file
        for on location {loc}
    ''')


def load_table(df, database_name, table_name):
    """ Write a table in associated catalog

    :params df: Dataframe to write
    :params database_name: namespace as in catalog
    :params table_name: table's name for reference
    :return: None
    """
    df.write.saveAsTable(f"{database_name}.{table_name}")
    print(f' Successfully added table {database_name}.{table_name} to catalog ')

def load_mock_db_initial_state(spark, df, table_name):
    """ Write mock initial mock data for dimension table
    This function creates mock initial load data that DMS would have
    generated for a dimensional table that stores names of all entities
    saved as Parquet files in tests/test_data. This will be used for
    unit tests as well as to load as a part of example ingestion job.
    :return: Location
    """

    location = get_location(
        actor='DMS',
        io='SOURCE',
        table_name=table_name,
        action='initial_state'
    )
    load_parquet(
        df=df,
        loc=location
    )

    print(f'''
        Successfully written Mock DB initial state
        for table {table_name} on location {location}
    ''')
    return location


In [17]:
def load_mock_dms_full_load(table_name, df):
    """ Mock AWS DMS full load
    :param table_name: Source Table Name
    :param df: Source Table Data
    """
    df_initial_load = (
        df
        .withColumn(
            "Op",
            lit('I')
        )
        .withColumn(
            'dms_export_timestamp',
            current_timestamp()
        )
    )
    location = get_location(
        actor='DMS',
        io='SINK',
        table_name=table_name,
        action='full_load'
    )
    load_parquet(
        df=df_initial_load,
        loc=location
    )
    print(f'''
        Successfully written Mock DMS full initial load
        for table {table_name} on location {location}
    ''')

    return location


In [18]:
def load_mock_dms_cdc_load(table_name, df):
    """ Mock AWS DMS full load
    :param table_name: Source Table Name
    :param df: Source Table Data
    """
    df_initial_load = (
        df
        .withColumn(
            "Op",
            lit('U')
        )
        .withColumn(
            'dms_export_timestamp',
            current_timestamp()
        )
    )
    location = get_location(
        actor='DMS',
        io='SINK',
        table_name=table_name,
        action='cdc_load'
    )
    load_parquet(
        df=df_initial_load,
        loc=location
    )
    print(f'''
        Successfully written Mock DMS full initial load
        for table {table_name} on location {location}
    ''')

    return location


In [19]:
def load_mock_dms_cdc_d_load(table_name, df):
    """ Mock AWS DMS full load
    :param table_name: Source Table Name
    :param df: Source Table Data
    """
    df_initial_load = (
        df
        .withColumn(
            "Op",
            lit('D')
        )
        .withColumn(
            'dms_export_timestamp',
            current_timestamp()
        )
    )
    location = get_location(
        actor='DMS',
        io='SINK',
        table_name=table_name,
        action='cdc_load'
    )
    load_parquet(
        df=df_initial_load,
        loc=location
    )
    print(f'''
        Successfully written Mock DMS full initial load
        for table {table_name} on location {location}
    ''')

    return location


### Creating the data

In [20]:
df_dimension_table = create_mock_dimension_table_dataframe(
    spark,
    1000
)



        dataframe of count 1000 
        was created from faker as 2024-07-15 11:07:44.444560
    


In [21]:
df_dimension_table.show(1, False, True)

-RECORD 0---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 name           | Sean Carrillo                                                                                                                                                                                                                                                     
 address        | 53169 Jeffrey Branch Suite 941\nChavezville, MS 81411                                                                                                                                                                                                             
 license_number | 547-YPVP                                                                                                                                               

### Writing data as parquet 

In [24]:
loc_db_initial_state = load_mock_db_initial_state(
    spark,
    df_dimension_table,
    TABLES[1]['name']
)



        Location: tests/test_data/db_data/demo/initial_state/2024_07_15-11:08:18_AM
            for DMS
            on SOURCE 
            for demo 
            on initial_state
            at 2024_07_15-11:08:18_AM
    



        Successfully written 1 file
        for on location tests/test_data/db_data/demo/initial_state/2024_07_15-11:08:18_AM
    

        Successfully written Mock DB initial state
        for table demo on location tests/test_data/db_data/demo/initial_state/2024_07_15-11:08:18_AM
    


### Mocking Database Migration Services output as parquet

In [26]:
df_mock_db_initial_state = extract_parquet(spark, loc_db_initial_state)



        dataframe of count 1000 '
        was read from location tests/test_data/db_data/demo/initial_state/2024_07_15-11:08:18_AM'
    


#### Full load

In [47]:
loc_mock_dms_data = load_mock_dms_full_load(
    TABLES[1]['name'],
    df_mock_db_initial_state
)


        Location: tests/test_data/dms_sink/demo/full_load/2024_07_15-01:01:01_PM
            for DMS
            on SINK 
            for demo 
            on full_load
            at 2024_07_15-01:01:01_PM
    

        Successfully written 1 file
        for on location tests/test_data/dms_sink/demo/full_load/2024_07_15-01:01:01_PM
    

        Successfully written Mock DMS full initial load
        for table demo on location tests/test_data/dms_sink/demo/full_load/2024_07_15-01:01:01_PM
    


#### CDC

##### mock updates

In [78]:
loc_mock_dms_data_u = load_mock_dms_cdc_load(
    TABLES[1]['name'],
    df_mock_db_initial_state
)


        Location: tests/test_data/dms_sink/demo/cdc_load/2024_07_15-01:25:44_PM
            for DMS
            on SINK 
            for demo 
            on cdc_load
            at 2024_07_15-01:25:44_PM
    

        Successfully written 1 file
        for on location tests/test_data/dms_sink/demo/cdc_load/2024_07_15-01:25:44_PM
    

        Successfully written Mock DMS full initial load
        for table demo on location tests/test_data/dms_sink/demo/cdc_load/2024_07_15-01:25:44_PM
    


##### mock deletes

In [49]:
loc_mock_dms_data_d = load_mock_dms_cdc_d_load(
    TABLES[1]['name'],
    df_mock_db_initial_state
)


        Location: tests/test_data/dms_sink/demo/cdc_load/2024_07_15-01:01:05_PM
            for DMS
            on SINK 
            for demo 
            on cdc_load
            at 2024_07_15-01:01:05_PM
    

        Successfully written 1 file
        for on location tests/test_data/dms_sink/demo/cdc_load/2024_07_15-01:01:05_PM
    

        Successfully written Mock DMS full initial load
        for table demo on location tests/test_data/dms_sink/demo/cdc_load/2024_07_15-01:01:05_PM
    


##### mock extracting data from parquet to dataframe

In [50]:
df_mock_dms_data = extract_parquet(spark, loc_mock_dms_data)



        dataframe of count 1000 '
        was read from location tests/test_data/dms_sink/demo/full_load/2024_07_15-01:01:01_PM'
    


In [51]:
df_mock_dms_data_d = extract_parquet(spark, loc_mock_dms_data_d)



        dataframe of count 1000 '
        was read from location tests/test_data/dms_sink/demo/cdc_load/2024_07_15-01:01:05_PM'
    


In [79]:
df_mock_dms_data_u = extract_parquet(spark, loc_mock_dms_data_u)



        dataframe of count 1000 '
        was read from location tests/test_data/dms_sink/demo/cdc_load/2024_07_15-01:25:44_PM'
    


### Creating tables in data lakehouse 

To be able to rerun the notebook several times, let's drop the table if it exists to start fresh.

In [55]:
%%sql

CREATE DATABASE IF NOT EXISTS prod_mirror

++
||
++
++

In [56]:
%%sql

DROP TABLE IF EXISTS prod_mirror.user_dimension 

++
||
++
++

##### Loading data from dataframe to minio data lakehouse

In [57]:
load_table(df_mock_dms_data, 'prod_mirror', 'user_dimension')

 Successfully added table prod_mirror.user_dimension to catalog 


In [58]:
%%sql

SELECT COUNT(*) as cnt
FROM prod_mirror.user_dimension

cnt
1000


In [59]:
%%sql

SELECT COUNT(*) as cnt, op
FROM prod_mirror.user_dimension
group by op

cnt,op
1000,I


##### Loading incremental data in lake house tables

In [60]:
df_mock_dms_data_d.writeTo("prod_mirror.user_dimension").append()



In [61]:
%%sql

SELECT COUNT(*) as cnt, op
FROM prod_mirror.user_dimension
group by op

cnt,op
1000,D
1000,I


In [80]:
df_mock_dms_data_u.writeTo("prod_mirror.user_dimension").append()


In [81]:
%%sql

SELECT COUNT(*) as cnt, op
FROM prod_mirror.user_dimension
group by op

cnt,op
3000,U
1000,I
1000,D


In [37]:
%%sql

DESCRIBE EXTENDED prod_mirror.user_dimension

col_name,data_type,comment
name,string,None
address,string,None
license_number,string,None
iban,string,None
bs,string,None
catch_phrase,string,None
company,string,None
paragraph,string,None
created_at,timestamp,None
updated_at,timestamp,None


In [64]:
%%sql

select count(*) from prod_mirror.user_dimension

count(1)
3000


note that this called a [change log table](https://tabular.io/blog/hello-world-of-cdc/)

## Solution 1: Change Log Table
In this we keep all incremental change in data as is

In [82]:
%%sql
WITH windowed_changes AS (
    SELECT
        op,
        uuid,
        updated_at,
        row_number() OVER (
            PARTITION BY uuid
            ORDER BY updated_at DESC) AS row_num
    FROM prod_mirror.user_dimension
),
mirror as (
    SELECT uuid, updated_at, op
    FROM windowed_changes WHERE row_num = 1 AND op != 'D'
)
select count(*), op from mirror group by op

count(1),op
1000,U


In [83]:
%%sql 
select uuid,op, dms_export_timestamp from prod_mirror.user_dimension where uuid ='2d762923-2cb9-4489-b8d8-23d18ad1aa8f'
order by dms_export_timestamp desc

uuid,op,dms_export_timestamp
2d762923-2cb9-4489-b8d8-23d18ad1aa8f,U,2024-07-15 13:25:44.160929
2d762923-2cb9-4489-b8d8-23d18ad1aa8f,D,2024-07-15 13:01:05.162668
2d762923-2cb9-4489-b8d8-23d18ad1aa8f,U,2024-07-15 13:01:03.921156
2d762923-2cb9-4489-b8d8-23d18ad1aa8f,U,2024-07-15 13:01:03.921156
2d762923-2cb9-4489-b8d8-23d18ad1aa8f,I,2024-07-15 13:01:01.919847


## Solution 2: View

In [ ]:
%%sql

DROP TABLE IF EXISTS prod_mirror.user_dimension_latest_view

In [85]:
%%sql 

INSERT INTO user_dimension_latest_view
SELECT
    uuid,
    current_date() AS creation_date,
    count(1) AS new_accounts
FROM prod_mirror.user_dimension a JOIN signups s ON s.acct_id = a.acct_id
WHERE a.created_on = current_date()
GROUP BY s.branch

UsageError: %%sql is a cell magic, but the cell body is empty. Did you mean the line magic %sql (single %)?


In [5]:
df = spark.read.parquet('tests/42.parquet')

In [7]:
df.collect()

24/07/20 14:21:49 ERROR Executor: Exception in task 0.0 in stage 4.0 (TID 4)/ 1]
java.lang.OutOfMemoryError: Java heap space
	at java.base/java.nio.HeapByteBuffer.<init>(HeapByteBuffer.java:61)
	at java.base/java.nio.ByteBuffer.allocate(ByteBuffer.java:348)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$getByteArrayRdd$2(SparkPlan.scala:368)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$getByteArrayRdd$2$adapted(SparkPlan.scala:368)
	at org.apache.spark.sql.execution.SparkPlan$$Lambda$2857/0x0000000101365040.apply(Unknown Source)
	at org.apache.spark.util.io.ChunkedByteBufferOutputStream.allocateNewChunkIfNeeded(ChunkedByteBufferOutputStream.scala:87)
	at org.apache.spark.util.io.ChunkedByteBufferOutputStream.write(ChunkedByteBufferOutputStream.scala:75)
	at net.jpountz.lz4.LZ4BlockOutputStream.flushBufferedData(LZ4BlockOutputStream.java:225)
	at net.jpountz.lz4.LZ4BlockOutputStream.write(LZ4BlockOutputStream.java:178)
	at java.base/java.io.DataOutputStream.write(DataOut

ConnectionRefusedError: [Errno 111] Connection refused

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


## Schema Evolution

Adding, dropping, renaming, or altering columns is easy and safe in Iceberg. In this example, we'll rename `fare_amount` to `fare` and `trip_distance` to `distance`. We'll also add a float column `fare_per_distance_unit` immediately after `distance`.

In [ ]:
%%sql

ALTER TABLE nyc.taxis RENAME COLUMN fare_amount TO fare

In [ ]:
%%sql

ALTER TABLE nyc.taxis RENAME COLUMN trip_distance TO distance

In [ ]:
%%sql

ALTER TABLE nyc.taxis ALTER COLUMN distance COMMENT 'The elapsed trip distance in miles reported by the taximeter.'

In [ ]:
%%sql

ALTER TABLE nyc.taxis ALTER COLUMN distance TYPE double;

In [ ]:
%%sql

ALTER TABLE nyc.taxis ALTER COLUMN distance AFTER fare;

In [ ]:
%%sql

ALTER TABLE nyc.taxis
ADD COLUMN fare_per_distance_unit float AFTER distance

Let's update the new `fare_per_distance_unit` to equal `fare` divided by `distance`.

In [ ]:
%%sql

UPDATE nyc.taxis
SET fare_per_distance_unit = fare/distance

In [ ]:
%%sql

SELECT
VendorID
,tpep_pickup_datetime
,tpep_dropoff_datetime
,fare
,distance
,fare_per_distance_unit
FROM nyc.taxis

## Expressive SQL for Row Level Changes
With Iceberg tables, `DELETE` queries can be used to perform row-level deletes. This is as simple as providing the table name and a `WHERE` predicate. If the filter matches an entire partition of the table, Iceberg will intelligently perform a metadata-only operation where it simply deletes the metadata for that partition.

Let's perform a row-level delete for all rows that have a `fare_per_distance_unit` greater than 4 or a `distance` greater than 2. This should leave us with relatively short trips that have a relatively high fare per distance traveled.

In [ ]:
%%sql

DELETE FROM nyc.taxis
WHERE fare_per_distance_unit > 4.0 OR distance > 2.0

There are some fares that have a `null` for `fare_per_distance_unit` due to the distance being `0`. Let's remove those as well.

In [ ]:
%%sql

DELETE FROM nyc.taxis
WHERE fare_per_distance_unit is null

In [ ]:
%%sql

SELECT
VendorID
,tpep_pickup_datetime
,tpep_dropoff_datetime
,fare
,distance
,fare_per_distance_unit
FROM nyc.taxis

In [ ]:
%%sql

SELECT COUNT(*) as cnt
FROM nyc.taxis

## Partitioning

A table’s partitioning can be updated in place and applied only to newly written data. Query plans are then split, using the old partition scheme for data written before the partition scheme was changed, and using the new partition scheme for data written after. People querying the table don’t even have to be aware of this split. Simple predicates in WHERE clauses are automatically converted to partition filters that prune out files with no matches. This is what’s referred to in Iceberg as *Hidden Partitioning*.

In [ ]:
%%sql

ALTER TABLE nyc.taxis
ADD PARTITION FIELD VendorID

## Metadata Tables

Iceberg tables contain very rich metadata that can be easily queried. For example, you can retrieve the manifest list for any snapshot, simply by querying the table's `snapshots` table.

In [ ]:
%%sql

SELECT snapshot_id, manifest_list
FROM nyc.taxis.snapshots

The `files` table contains loads of information on data files, including column level statistics such as null counts, lower bounds, and upper bounds.

In [ ]:
%%sql

SELECT file_path, file_format, record_count, null_value_counts, lower_bounds, upper_bounds
FROM nyc.taxis.files

## Time Travel

The history table lists all snapshots and which parent snapshot they derive from. The `is_current_ancestor` flag let's you know if a snapshot is part of the linear history of the current snapshot of the table.

In [ ]:
%%sql

SELECT *
FROM nyc.taxis.history

You can time-travel by altering the `current-snapshot-id` property of the table to reference any snapshot in the table's history. Let's revert the table to it's original state by traveling to the very first snapshot ID.

In [ ]:
%%sql --var df

SELECT *
FROM nyc.taxis.history

In [ ]:
original_snapshot = df.head().snapshot_id
spark.sql(f"CALL system.rollback_to_snapshot('nyc.taxis', {original_snapshot})")
original_snapshot

In [ ]:
%%sql

SELECT
VendorID
,tpep_pickup_datetime
,tpep_dropoff_datetime
,fare
,distance
,fare_per_distance_unit
FROM nyc.taxis

Another look at the history table shows that the original state of the table has been added as a new entry
with the original snapshot ID.

In [ ]:
%%sql

SELECT *
FROM nyc.taxis.history

In [ ]:
%%sql

SELECT COUNT(*) as cnt
FROM nyc.taxis